In [ ]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

## Store Part I results into DataFrame

##### Load the csv exported in Part I to a DataFrame

In [ ]:
weather_csv_file = "output/City_Weather_Data.csv"

weather_df = pd.read_csv(weather_csv_file)

weather_df

### Humidity Heatmap

##### Configure gmaps.
##### Use the Lat and Lng as locations and Humidity as the weight.
##### Add Heatmap layer to map.

In [ ]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store and drop NaN values
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Lat", "Lng"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

## Create new DataFrame fitting weather criteria

##### Narrow down the cities to fit weather conditions.
##### Drop any rows will null values.

In [ ]:
# Narrow down the DataFrame to find your ideal weather condition.
criteria_weather_df = weather_df

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# A max temperature lower than 295 degrees but higher than 290.
criteria_weather_df = criteria_weather_df.loc[(criteria_weather_df["Max Temp"] < 295) & (criteria_weather_df["Max Temp"] > 290)]

# Wind speed less than 10 mph.
criteria_weather_df = criteria_weather_df.loc[criteria_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
criteria_weather_df = criteria_weather_df.loc[criteria_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
criteria_weather_df = criteria_weather_df.dropna()

criteria_weather_df

## Hotel Map

##### Store into variable named hotel_df.
##### Add a "Hotel Name" column to the DataFrame.
##### Set parameters to search for hotels with 5000 meters.
##### Hit the Google Places API for each city's coordinates.
##### Store the first Hotel result into the DataFrame.Plot markers on top of the heatmap.

In [ ]:
hotel_df = criteria_weather_df

# Set parameters to search for hotels with 5000 meters.
set_params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng
    lat = row["Lat"]
    lng = row["Lng"]

    # Hit the Google Places API for each city's coordinates.
    set_params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    print(f" Row {index}: {row['City']}.")
    response = requests.get(base_url, params=set_params).json()
    
    results = response['results']
    
    #Add a "Hotel Name" column to the DataFrame.
    try:
        print(f" Hotel name is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Hotel Missing")


In [ ]:
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations)
fig.add_layer(marker)

# Display figure
fig